**Problem 6**

Write an elementary Python program that creates a present value, future value “calculator.” The program
should query the user for the following:

- Present value or future value calculation
- Number of “discrete forecast” periods
- Number of distinct “growth” phases (including “none”), and for each phase:
    - Applicable growth rate
    - Length of phase
    - And, if it is the last growth phase, whether it is an annuity or perpetuity (only the last phase
    can be either an annuity or perpetuity; all others must be annuities)
- Applicable discount rate = r
- And then return the appropriate present value or future value
- Recommendation: It might be a good idea to show how it was constructed, for example:
    - PV Discrete Forecast: $100
    - PV Phase I Annuity: $80
    - PV Phase II Annuity: $70
    - PV Phase III Perpetuity: $500
    - TOTAL PV = $750
- HINT: If you are using deferred annuity or perpetuity formulas, you need to take the PV of the PV
of the deferred annuity or perpetuity.


**Full Problem 6 Code**

*Assumptions*

- Discrete values at period 0 are not included in PV calculation as they are not future and assumed to have already been paid
- If you want to find the FV of a single cash flow (that is already assumed to be paid), enter it as discrete at period 0
- The calculator assumes a 2 stage or higher model, starting with discrete cash flows, then starting growth phases from the last discrete period
- Perpetuities must be the final growth phase


In [111]:
def pvHelper(tempPV, g_start, dr):
    tempPV = tempPV * (1 / ((1 + dr) ** (g_start-1)))
    return tempPV

def presentValue(dis_p, discrete_dict, dr, growth_phases, phase_dict):
    totalPV = 0
    last_discrete_value = 0
    last_period = 0

    
    for x in range(dis_p):
        currentVal = discrete_dict[x + 1]["value"]
        t = discrete_dict[x + 1]["period"]

        tempPV = currentVal * (1 / (1 + dr) ** t)
        tempPV = round(tempPV, 2)
        last_discrete_value = currentVal  
        last_period = t  

        if (t == 0):
            continue

        print("PV Discrete Forecast " + str(x + 1) + ": " + str(tempPV))
        totalPV += tempPV

        
   
    for y in range(growth_phases):
        g_rate = phase_dict[y + 1]["growth_rate"]
        g_length = phase_dict[y + 1]["length"]
        last_phase = phase_dict[y + 1]["last_phase"]

        g_start = last_period + 1
        g_end = g_start + g_length
        

        if y == 0:
            g_val = last_discrete_value 
        else:
            g_val = phase_dict[y]["value"]  
            

        phase_dict[y + 1]["value"] = g_val * (1 + g_rate) ** g_length
        

        if last_phase == 2:
            tempPV = (g_val*(1+g_rate)) / (dr - g_rate)
            if g_start != 1:
                tempPV = pvHelper(tempPV, g_start, dr)
            tempPV = round(tempPV, 2)
            print("PV Growth Phase " + str(y + 1) + " Perpetuity: " + str(tempPV))
            totalPV += tempPV
        else:
            #tempPV = (g_val * (1 - ((1 + g_rate) / (1 + dr)) ** g_length) / (dr - g_rate))
            tempPV = ((g_val * (1+g_rate))/(dr-g_rate))*(1-((1+g_rate)/(1+dr))**g_length)
            if g_start != 1:
                tempPV = pvHelper(tempPV, g_start, dr)
            totalPV += tempPV
            tempPV = round(tempPV, 2)

            print("PV Growth Phase " + str(y + 1) + " Annuity: " + str(tempPV))
            
        
        last_period = g_end - 1  # Update last period to the end of the current growth phase

    return totalPV

def futureValueFromPV(pv, dr, fv_period, zero_period):
    if (zero_period != 0 and pv == 0):
        return zero_period * ((1 + dr) ** fv_period)
    return pv * ((1 + dr) ** fv_period)

def get_valid_input(prompt, input_type):
    while True:
        try:
            value = input_type(input(prompt))
            return value
        except ValueError:
            print(f"Invalid input! Please enter a valid {input_type.__name__}.")

def pv_fv_calculator():
    while True:
        calcType = input("Please enter the calculation type, 'PV' for present value or 'FV' for future value: ")
        if calcType == 'FV' or calcType == 'PV':
            break
        else:
            print("Invalid calculation type! Try again.")

    dr = get_valid_input("What is the discount rate? ", float)
    dis_p = get_valid_input("How many discrete forecast periods? ", int)
    discrete_dict = dict()
    zero_period = 0
    for d in range(dis_p):
        d_period = get_valid_input(f"For discrete phase {d + 1}, enter the period of the cash flow: ", int)
        d_value = get_valid_input(f"For discrete phase {d + 1}, enter the value of the cash flow at that period: ", float)
        if (d_period == 0):
            zero_period = d_value
        discrete_dict[d + 1] = {
            "period": d_period,
            "value": d_value
        }

    growth_phases = get_valid_input("How many distinct growth phases? ", int)
    phase_dict = dict()
    for x in range(growth_phases):
        g_rate = get_valid_input(f"For phase {x + 1}, enter the growth rate: ", float)
        g_length = get_valid_input(f"For phase {x + 1}, enter the length of the phase (enter -1 for perpetuity): ", int)
        last_phase = 0
        if x + 1 == growth_phases:
            while True:
                last_phase = input("Is the last phase an annuity or perpetuity? ").lower()
                if last_phase == "annuity":
                    last_phase = 1
                    break
                elif last_phase == "perpetuity":
                    last_phase = 2
                    break
                else:
                    print("Invalid input! Please enter 'annuity' or 'perpetuity'.")
        phase_dict[x + 1] = {
            "growth_rate": g_rate,
            "length": g_length,
            "value": 0,  # This will be calculated based on previous phases
            "last_phase": last_phase
        }

    if calcType == 'PV':
        pv = presentValue(dis_p, discrete_dict, dr, growth_phases, phase_dict)
        pv = round(pv, 2)
        print("TOTAL PV: " + str(pv))

    if calcType == 'FV':
        fv_period = get_valid_input("Which period would you like the future value for? ", int)
        pv = presentValue(dis_p, discrete_dict, dr, growth_phases, phase_dict)
        fv = futureValueFromPV(pv, dr, fv_period, zero_period)
        fv = round(fv, 2)
        print("TOTAL FV: " + str(fv))

# Run the calculator
pv_fv_calculator()


PV Growth Phase 1 Annuity: 4384.42
PV Growth Phase 2 Perpetuity: 7570.33
TOTAL PV: 11954.75
